# SageMaker Studio Domainを削除

* ユーザーのアプリケーションを削除する
* ユーザーを削除する
* ドメインを削除する

以下も検討
* EFSを削除
* spacesを削除
* spacesの共有EFSを削除

https://docs.aws.amazon.com/sagemaker/latest/dg/gs-studio-delete-domain.html


In [1]:
import boto3
sm = boto3.Session().client('sagemaker')

In [2]:
boto3.__version__

'1.23.10'

# 0. IAM要件

ドメイン削除
* sagemaker:DeleteDomain

## delete user application

In [3]:
response = sm.list_domains()

In [6]:
response['Domains'][0]

{'DomainArn': 'arn:aws:sagemaker:us-west-2:805433377179:domain/d-9bozig4je0oq',
 'DomainId': 'd-9bozig4je0oq',
 'DomainName': 'domein2',
 'Status': 'InService',
 'CreationTime': datetime.datetime(2022, 12, 7, 6, 45, 14, 923000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2022, 12, 7, 6, 48, 1, 564000, tzinfo=tzlocal()),
 'Url': 'https://d-9bozig4je0oq.studio.us-west-2.sagemaker.aws'}

https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html#SageMaker.Client.list_user_profiles

In [8]:
res_list_user = sm.list_user_profiles(
    SortOrder='Ascending',
    SortBy='CreationTime',
    DomainIdEquals=response['Domains'][0]['DomainId'],
    #UserProfileNameContains='string'
)

In [9]:
res_list_user

{'UserProfiles': [{'DomainId': 'd-9bozig4je0oq',
   'UserProfileName': 'default-1670395454974',
   'Status': 'InService',
   'CreationTime': datetime.datetime(2022, 12, 7, 6, 48, 20, 824000, tzinfo=tzlocal()),
   'LastModifiedTime': datetime.datetime(2022, 12, 7, 6, 48, 22, 803000, tzinfo=tzlocal())}],
 'ResponseMetadata': {'RequestId': '812fc961-5f27-4352-883b-169aacf3bb65',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '812fc961-5f27-4352-883b-169aacf3bb65',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '179',
   'date': 'Sat, 10 Dec 2022 02:37:20 GMT'},
  'RetryAttempts': 0}}

https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html#SageMaker.Client.list_apps

In [13]:
res_list_apps = sm.list_apps(
    SortOrder='Ascending',
    SortBy='CreationTime',
    DomainIdEquals=response['Domains'][0]['DomainId'],
    UserProfileNameEquals=res_list_user['UserProfiles'][0]['UserProfileName']
    #SpaceNameEquals='string'
)

In [14]:
res_list_apps

{'Apps': [],
 'ResponseMetadata': {'RequestId': '7b9c5ed4-5312-4724-828f-bc6e5db34d8c',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '7b9c5ed4-5312-4724-828f-bc6e5db34d8c',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '11',
   'date': 'Sat, 10 Dec 2022 02:40:19 GMT'},
  'RetryAttempts': 0}}

アプリなかったので後で書く

### delete user

https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html#SageMaker.Client.delete_user_profile

In [15]:
sm.delete_user_profile(
    DomainId=response['Domains'][0]['DomainId'],    
    UserProfileName=res_list_user['UserProfiles'][0]['UserProfileName']
)

ClientError: An error occurred (AccessDeniedException) when calling the DeleteUserProfile operation: User: arn:aws:sts::805433377179:assumed-role/AmazonSageMaker-ExecutionRole-20200428T181604/SageMaker is not authorized to perform: sagemaker:DeleteUserProfile on resource: arn:aws:sagemaker:us-west-2:805433377179:user-profile/d-9bozig4je0oq/default-1670395454974 because no identity-based policy allows the sagemaker:DeleteUserProfile action

権限が足りないようだ。シェルスクリプトでアドミンが実行した方がいいかもしれない。

# delete domain

https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html#SageMaker.Client.delete_domain

ドメインIDを取得

https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html#SageMaker.Client.list_domains

In [5]:
response = sm.list_domains()

In [10]:
response['Domains'][0]['DomainId']

'd-9bozig4je0oq'

In [12]:
response = sm.delete_domain(
    DomainId=response['Domains'][1]['DomainId'],
    RetentionPolicy={
        'HomeEfsFileSystem': 'Delete'
    }
)

ClientError: An error occurred (AccessDeniedException) when calling the DeleteDomain operation: User: arn:aws:sts::805433377179:assumed-role/AmazonSageMaker-ExecutionRole-20200428T181604/SageMaker is not authorized to perform: sagemaker:DeleteDomain on resource: arn:aws:sagemaker:us-west-2:805433377179:domain/d-fraeraxty77v because no identity-based policy allows the sagemaker:DeleteDomain action

# END

In [ ]:
def cleanup_boto3(experiment_name):
    trials = sm.list_trials(ExperimentName=experiment_name)['TrialSummaries']
    print('TrialNames:')
    for trial in trials:
        trial_name = trial['TrialName']
        print(f"\n{trial_name}")

        components_in_trial = sm.list_trial_components(TrialName=trial_name)
        print('\tTrialComponentNames:')
        for component in components_in_trial['TrialComponentSummaries']:
            component_name = component['TrialComponentName']
            print(f"\t{component_name}")
            sm.disassociate_trial_component(TrialComponentName=component_name, TrialName=trial_name)
            try:
                # comment out to keep trial components
                sm.delete_trial_component(TrialComponentName=component_name)
            except:
                # component is associated with another trial
                continue
            # to prevent throttling
            time.sleep(.5)
        sm.delete_trial(TrialName=trial_name)
    sm.delete_experiment(ExperimentName=experiment_name)
    print(f"\nExperiment {experiment_name} deleted")

In [ ]:
for experiment_name in ['mlops-experiment01']:
    print(experiment_name)
    cleanup_boto3(experiment_name)

### experiments名を取得
https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html#SageMaker.Client.list_experiments

In [ ]:
### experiments名一覧を出力
for i in range(len(sm.list_experiments()['ExperimentSummaries'])):
    print(sm.list_experiments()['ExperimentSummaries'][i]['ExperimentName'])